# Using tf.layer

https://www.tensorflow.org/programmers_guide/datasets

In [1]:
from PIL import Image
import numpy as np
import tensorflow as tf

/Users/tjw/anaconda3/envs/keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## MNIST Dataset

In [2]:
import lzma
import pickle
with lzma.open("mnist.pkl.xz", 'rb') as f:
    train_set, validation_set, test_set = pickle.load(f, encoding='latin1')

In [3]:
train_X, train_y = train_set
validation_X, validation_y = validation_set
test_X, test_y = test_set
train_Y = np.eye(10)[train_y]
test_Y = np.eye(10)[test_y]
validation_Y = np.eye(10)[validation_y]

In [4]:
from IPython.display import display
def showX(X):
    int_X = (X*255).clip(0,255).astype('uint8')
    # N*784 -> N*28*28 -> 28*N*28 -> 28 * 28N
    int_X_reshape = int_X.reshape(-1,28,28).swapaxes(0,1).reshape(28,-1)
    display(Image.fromarray(int_X_reshape))
# 訓練資料， X 的前 20 筆
showX(train_X[:20])
print(train_y)

[5 0 4 ... 8 4 8]


### using dataset

In [5]:
training_data = tf.data.Dataset.from_tensor_slices((train_X, train_Y)).shuffle(buffer_size=10000).batch(40)
validation_data = tf.data.Dataset.from_tensor_slices((validation_X, validation_Y)).batch(40)


iterator = tf.data.Iterator.from_structure(training_data.output_types,
                                           training_data.output_shapes)

training_init_op = iterator.make_initializer(training_data)
validation_init_op = iterator.make_initializer(validation_data)

X, Y_ = iterator.get_next()

In [6]:
# 設定 cnn 的 layers

X_image = tf.reshape(X, [-1, 28, 28, 1])

# fisrt layer
h_conv1 = tf.layers.conv2d(X_image, 32, 3, padding='same', activation=tf.nn.relu)
h_pool1 = tf.layers.max_pooling2d(h_conv1, 2, 2)

# second layer
h_conv2 = tf.layers.conv2d(h_pool1, 64, 3, padding='same', activation=tf.nn.relu)
h_pool2 = tf.layers.max_pooling2d(h_conv2, 2, 2)
h_pool2_flat = tf.layers.flatten(h_pool2)

# fully-connected layer
h_fc = tf.layers.dense(h_pool2_flat, 1024, activation=tf.nn.relu, name='fc')

# Dropout
with tf.name_scope('dropout'):
    keep_prob = tf.placeholder("float", name="keep_prob")
    h_fc_drop = tf.nn.dropout(h_fc, keep_prob)

# Readout
Y = tf.layers.dense(h_fc_drop, 10, activation=tf.nn.relu, name='readout')

cross_entropy =  tf.losses.softmax_cross_entropy(onehot_labels=Y_, logits=Y)
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
prediction = tf.argmax(Y, 1, name="prediction")
correct_prediction = tf.equal(prediction, tf.argmax(Y_, 1), name="correction")
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")

In [7]:
import time
sess = tf.Session()
sess.run(tf.global_variables_initializer())
t0 = time.time()
for epoch in range(3):
    sess.run(training_init_op)
    while True:
        try:
            sess.run(train_step, {keep_prob: 0.5 })
        except tf.errors.OutOfRangeError:
            print("End of epoch", epoch, "time:", time.time()-t0)
            break
    sess.run(validation_init_op)
    validation_accuracy = np.mean([sess.run(accuracy,{keep_prob: 1.0 }) for i in range(10)])
    print("Epoch %d, validation accuracy %g"%(epoch, validation_accuracy))
sess.close()

End of epoch 0 time: 57.661760091781616
Epoch 0, validation accuracy 0.9575
End of epoch 1 time: 110.42608594894409
Epoch 1, validation accuracy 0.9725
End of epoch 2 time: 160.75373792648315
Epoch 2, validation accuracy 0.975
End of epoch 3 time: 210.8800938129425
Epoch 3, validation accuracy 0.9775
End of epoch 4 time: 260.91600370407104
Epoch 4, validation accuracy 0.9775
End of epoch 5 time: 311.07409477233887
Epoch 5, validation accuracy 0.985
End of epoch 6 time: 361.27688789367676
Epoch 6, validation accuracy 0.985
End of epoch 7 time: 411.6825008392334
Epoch 7, validation accuracy 0.985
End of epoch 8 time: 465.17056369781494
Epoch 8, validation accuracy 0.9925
End of epoch 9 time: 520.994081735611
Epoch 9, validation accuracy 0.9875
